In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import utils
import torchvision
from torchvision.io import image
from torchvision.transforms.functional import to_pil_image
import pandas as pd
import time
import numpy as np
import collections
import re
import random
import math

In [2]:
batch_size, num_steps = 32, 35
# train_iter, vocab = utils.load_data_txt(utils.sanguo_txt_path, batch_size, num_steps)
train_iter, vocab = utils.load_data_txt(utils.santi_txt_path, batch_size, num_steps)
print(len(vocab))

1464


In [3]:
def get_lstm_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device)*0.01

    def init_three_params():
        return (normal((num_inputs, num_hiddens)), normal((num_hiddens, num_hiddens)), torch.zeros(num_hiddens, device=device))

    W_xi, W_hi, b_i = init_three_params()  # 输入门参数
    W_xf, W_hf, b_f = init_three_params()  # 遗忘门参数
    W_xo, W_ho, b_o = init_three_params()  # 输出门参数
    W_xc, W_hc, b_c = init_three_params()  # 候选记忆元参数
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [4]:
# 初始化状态（隐状态+候选记忆元）
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), torch.zeros((batch_size, num_hiddens), device=device))

In [5]:
# 定义模型
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid((X @ W_xi) + (H @ W_hi) + b_i)
        F = torch.sigmoid((X @ W_xf) + (H @ W_hf) + b_f)
        O = torch.sigmoid((X @ W_xo) + (H @ W_ho) + b_o)
        C_tilda = torch.tanh((X @ W_xc) + (H @ W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * torch.tanh(C)
        Y = (H @ W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, C)

In [6]:
# predict_prefix = ['cao cao', 'kingdom']
predict_prefix = ['三体组织', '物理学']

In [7]:
vocab_size, num_hiddens, device = len(vocab), 256, utils.try_gpu()
num_epochs, lr = 500, 1
model = utils.RNNModelScratch(len(vocab), num_hiddens, device, get_lstm_params, init_lstm_state, lstm)
utils.train_ch8(model, train_iter, vocab, lr, num_epochs, device, predict_prefix=predict_prefix)

三体组织的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 50/500, ppl: 484.24730154413913
三体组织，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 100/500, ppl: 482.5993570773085
三体组织，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 150/500, ppl: 475.03783388234984
三体组织，，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的的，，的的
epoch: 200/500, ppl: 461.7112383994697
三体组织，我是的，这，我的的，这，我是的的，这

# 简洁实现

In [8]:
num_inputs = vocab_size
lstm_layer = nn.LSTM(num_inputs, num_hiddens)
model = utils.RNNModel(lstm_layer, len(vocab))
model = model.to(device)
utils.train_ch8(model, train_iter, vocab, lr, num_epochs, device, predict_prefix=predict_prefix)

三体组织的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 50/500, ppl: 484.3971834472492
三体组织，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
epoch: 100/500, ppl: 468.0195082052079
三体组织的，，我，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
epoch: 150/500, ppl: 407.5926265892861
三体组织，但是个，这的，，是是，这的，，是在的，，我们的，，是是，，我的，，我们的，，是是，，我的，，我们的，，是是，，不的，，是在，，我的，，我的，，我们的，，是是，，不的，，比的，，我们的，，是是，，不的，，是在，，我的，，我的，，我的，，我们的，，是是，，不的，，比的，，我们的，，是是，，我的，，我是，，我的，，是，这个，，是是，，我的，，我的，，我们的，，是是，，不的，，比的，，我们的，，是是，，
epoch: 200/500, ppl: 305.58618952588273
三体组织的，他们的一个的，但是是一个择的，我是在

# 长短期记忆网络有三种类型的门：输入门、遗忘门和输出门。

# 长短期记忆网络的隐藏层输出包括“隐状态”和“记忆元”。只有隐状态会传递到输出层，而记忆元完全属于内部信息。

# 长短期记忆网络可以缓解梯度消失和梯度爆炸。